In [1]:
include("../output.jl")
using ImpvolOutput, FileIO
results = load("../experiments/baseline/actual/results.jld2")["results"]
parameters = load("../experiments/baseline/common_parameters.jld2")["parameters"]
real_GDP_model = sum(ImpvolOutput.make_series(results, :real_GDP), 3)

1×25×1×36 Array{Float64,4}:
[:, :, 1, 1] =
 63833.9  31849.2  38829.1  95835.4  84047.2  …  37373.6  204752.0  1.13267e6

[:, :, 1, 2] =
 66276.9  31205.5  41873.4  1.04486e5  …  38404.3  2.13369e5  1.20129e6

[:, :, 1, 3] =
 63192.2  32279.3  41238.0  1.11372e5  …  40113.3  2.07413e5  1.18411e6

...

[:, :, 1, 34] =
 1.68904e5  74876.7  92891.8  2.7833e5  …  83422.9  511608.0  3.47262e6

[:, :, 1, 35] =
 1.8557e5  81132.4  93230.6  3.06143e5  …  89085.8  5.5663e5  3.65406e6

[:, :, 1, 36] =
 2.05685e5  89386.5  1.04464e5  3.69934e5  …  95611.0  624110.0  3.92774e6

In [2]:
rows = [("1970s", 1:8), ("1980s", 9:18), ("1990s", 19:28), ("2000s", 28:36)]
columns = ["actual", "kappa1972", "nosectoral", "nosectoral_kappa1972"]

4-element Array{String,1}:
 "actual"              
 "kappa1972"           
 "nosectoral"          
 "nosectoral_kappa1972"

In [3]:
function get_volatility(rows, column)
    results = load("../experiments/baseline/$(column)/results.jld2")["results"]
    real_GDP_model = sum(ImpvolOutput.make_series(results, :real_GDP), 3)
    return [mean(ImpvolOutput.calculate_volatilities(real_GDP_model, parameters, true, row[2])) for row in rows]
end

get_volatility (generic function with 1 method)

In [4]:
using DataFrames
volatility = DataFrame()
volatility[:decades] = [row[1] for row in rows]
for column in columns
    volatility[Symbol(column)] = get_volatility(rows, column)
end


In [5]:
volatility

,decades,actual,kappa1972,nosectoral,nosectoral_kappa1972
1,1970s,0.000847938,0.000878369,0.00111567,0.00125421
2,1980s,0.0010269,0.0011704,0.00118609,0.00160623
3,1990s,0.00106483,0.0016131,0.00180111,0.00286829
4,2000s,0.000741105,0.00224316,0.00176957,0.00321772


In [6]:
stats = volatility
stats[:trade_barriers] = 100 * (stats[:actual] - stats[:kappa1972]) ./ stats[:kappa1972]
stats[:diversification] = 100 * (stats[:nosectoral] - stats[:nosectoral_kappa1972]) ./ stats[:kappa1972]
stats[:specialization] = 100 * (stats[:actual] - stats[:kappa1972] - stats[:nosectoral] + stats[:nosectoral_kappa1972]) ./ stats[:kappa1972]


4-element Array{Float64,1}:
 12.3083 
 23.6371 
 32.1689 
 -2.40302

In [7]:
for col in columns
    delete!(volatility, Symbol(col))
end

In [8]:
using CSV
CSV.write("../volatility_by_decade.csv", volatility)

CSV.Sink{Void,DataType}(    CSV.Options:
        delim: ','
        quotechar: '"'
        escapechar: '\\'
        missingstring: ""
        dateformat: nothing
        decimal: '.'
        truestring: 'true'
        falsestring: 'false'
        internstrings: true, IOBuffer(data=UInt8[...], readable=true, writable=true, seekable=true, append=false, size=0, maxsize=Inf, ptr=1, mark=-1), "../volatility_by_decade.csv", 54, true, String["decades", "trade_barriers", "diversification", "specialization"], 4, false, Val{false})